In [ ]:
!pip install fiftyone

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%%shell

# Download TorchVision repo to use some files from
# references/detection
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.3.0

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

fatal: destination path 'vision' already exists and is not an empty directory.
HEAD is now at be376084d version check against PyTorch's CUDA version


In [ ]:
import torch

torch.manual_seed(1)

In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz

In [ ]:
fo_dataset = foz.load_zoo_dataset("coco-2017", "validation")

INFO:fiftyone.zoo.datasets:Downloading split 'validation' to '/root/fiftyone/coco-2017/validation' if necessary


Found annotations at '/root/fiftyone/coco-2017/raw/instances_val2017.json'


INFO:fiftyone.utils.coco:Found annotations at '/root/fiftyone/coco-2017/raw/instances_val2017.json'


Images already downloaded


INFO:fiftyone.utils.coco:Images already downloaded


Existing download of split 'validation' is sufficient


INFO:fiftyone.zoo.datasets:Existing download of split 'validation' is sufficient


Loading existing dataset 'coco-2017-validation'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


INFO:fiftyone.zoo.datasets:Loading existing dataset 'coco-2017-validation'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


We will be needing the height and width of images later in this notebook so we need to compute metadata on our dataset.

In [ ]:
fo_dataset.compute_metadata()

We can create a session and visualize this dataset in the [FiftyOne App](https://voxel51.com/docs/fiftyone/user_guide/app.html).

In [ ]:
session = fo.launch_app(fo_dataset)

In [ ]:
import torch
import fiftyone.utils.coco as fouc
from PIL import Image


class FiftyOneTorchDataset(torch.utils.data.Dataset):
    def __init__(
        self,
        fiftyone_dataset,
        transforms=None,
        gt_field="ground_truth",
        classes=None,
    ):
        self.samples = fiftyone_dataset
        self.transforms = transforms
        self.gt_field = gt_field

        self.img_paths = self.samples.values("filepath")

        self.classes = classes
        if not self.classes:
            # Get list of distinct labels that exist in the view
            self.classes = self.samples.distinct(
                "%s.detections.label" % gt_field
            )

        if self.classes[0] != "background":
            self.classes = ["background"] + self.classes

        self.labels_map_rev = {c: i for i, c in enumerate(self.classes)}

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        sample = self.samples[img_path]
        metadata = sample.metadata
        img = Image.open(img_path).convert("RGB")

        boxes = []
        labels = []
        area = []
        iscrowd = []
        detections = sample[self.gt_field].detections
        for det in detections:
            category_id = self.labels_map_rev[det.label]
            coco_obj = fouc.COCOObject.from_label(
                det, metadata, category_id=category_id,
            )
            x, y, w, h = coco_obj.bbox
            boxes.append([x, y, x + w, y + h])
            labels.append(coco_obj.category_id)
            area.append(coco_obj.area)
            iscrowd.append(coco_obj.iscrowd)

        target = {}
        target["boxes"] = torch.as_tensor(boxes, dtype=torch.float32)
        target["labels"] = torch.as_tensor(labels, dtype=torch.int64)
        target["image_id"] = torch.as_tensor([idx])
        target["area"] = torch.as_tensor(area, dtype=torch.float32)
        target["iscrowd"] = torch.as_tensor(iscrowd, dtype=torch.int64)

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.img_paths)

    def get_classes(self):
        return self.classes

In [ ]:
from fiftyone import ViewField as F

detec_list = ["person", "dog"]
detec_view = fo_dataset.filter_labels("ground_truth",
        F("label").is_in(detec_list))

print(len(detec_view))

2794


In [ ]:
session.view = detec_view

In [ ]:
# From the torchvision references we cloned
import transforms as T

train_transforms = T.Compose([T.ToTensor(), T.RandomHorizontalFlip(0.5)])

In [ ]:
# use our dataset and defined transformations
torch_dataset = FiftyOneTorchDataset(detec_view, train_transforms,
        classes=detec_list)

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

def get_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

In [ ]:
# Import functions from the torchvision references we cloned
from engine import train_one_epoch, evaluate
import utils


def train(model, dataset,  num_epochs=4):
  # define training and validation data loaders
  data_loader = torch.utils.data.DataLoader(
  dataset, batch_size=2, shuffle=True, num_workers=2,
  collate_fn=utils.collate_fn)
  # check if gpu is availabe , otherwise use cpu
  device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
  print("Using device %s" % device)
  model.to(device)

 
  steps_per_epoch = len(data_loader)
  optimizer = torch.optim.Adam(model.parameters())
  for epoch in range(num_epochs):
    train_one_epoch(model, optimizer, data_loader, device,epoch+1, 100)
  optimizer.step()

In [ ]:
model = get_model(len(detec_list)+1)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
train(model, torch_dataset)

Using device cuda


/usr/local/lib/python3.7/dist-packages/pymongo/topology.py:165: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: https://pymongo.readthedocs.io/en/stable/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "
/usr/local/lib/python3.7/dist-packages/pymongo/topology.py:165: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: https://pymongo.readthedocs.io/en/stable/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "


Epoch: [0]  [   0/1397]  eta: 1:25:54  lr: 0.000002  loss: 1.6718 (1.6718)  loss_classifier: 1.3732 (1.3732)  loss_box_reg: 0.2600 (0.2600)  loss_objectness: 0.0313 (0.0313)  loss_rpn_box_reg: 0.0074 (0.0074)  time: 3.6897  data: 0.4088  max mem: 2337
Epoch: [0]  [ 100/1397]  eta: 0:14:01  lr: 0.000102  loss: 0.3387 (0.6483)  loss_classifier: 0.1209 (0.2855)  loss_box_reg: 0.2113 (0.2973)  loss_objectness: 0.0137 (0.0370)  loss_rpn_box_reg: 0.0125 (0.0284)  time: 0.6330  data: 0.0087  max mem: 4181
Epoch: [0]  [ 200/1397]  eta: 0:12:27  lr: 0.000202  loss: 0.4755 (0.5977)  loss_classifier: 0.1854 (0.2377)  loss_box_reg: 0.2199 (0.2708)  loss_objectness: 0.0823 (0.0510)  loss_rpn_box_reg: 0.0348 (0.0382)  time: 0.6128  data: 0.0096  max mem: 4182
Epoch: [0]  [ 300/1397]  eta: 0:11:23  lr: 0.000302  loss: 0.3611 (0.5858)  loss_classifier: 0.1217 (0.2181)  loss_box_reg: 0.1557 (0.2624)  loss_objectness: 0.0337 (0.0615)  loss_rpn_box_reg: 0.0180 (0.0438)  time: 0.6133  data: 0.0094  max me

/usr/local/lib/python3.7/dist-packages/pymongo/topology.py:165: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: https://pymongo.readthedocs.io/en/stable/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "
/usr/local/lib/python3.7/dist-packages/pymongo/topology.py:165: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: https://pymongo.readthedocs.io/en/stable/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "


Epoch: [1]  [   0/1397]  eta: 0:17:27  lr: 0.001000  loss: 0.3927 (0.3927)  loss_classifier: 0.1436 (0.1436)  loss_box_reg: 0.1232 (0.1232)  loss_objectness: 0.1016 (0.1016)  loss_rpn_box_reg: 0.0243 (0.0243)  time: 0.7497  data: 0.2216  max mem: 4553
Epoch: [1]  [ 100/1397]  eta: 0:13:05  lr: 0.001000  loss: 0.5653 (0.5235)  loss_classifier: 0.1728 (0.1560)  loss_box_reg: 0.1666 (0.1598)  loss_objectness: 0.0823 (0.1366)  loss_rpn_box_reg: 0.0331 (0.0711)  time: 0.6246  data: 0.0082  max mem: 4553
Epoch: [1]  [ 200/1397]  eta: 0:12:03  lr: 0.001000  loss: 0.4225 (0.5396)  loss_classifier: 0.1389 (0.1659)  loss_box_reg: 0.1087 (0.1735)  loss_objectness: 0.0765 (0.1294)  loss_rpn_box_reg: 0.0243 (0.0708)  time: 0.6345  data: 0.0088  max mem: 4553
Epoch: [1]  [ 300/1397]  eta: 0:11:05  lr: 0.001000  loss: 0.5222 (0.5302)  loss_classifier: 0.1206 (0.1603)  loss_box_reg: 0.1087 (0.1671)  loss_objectness: 0.1362 (0.1282)  loss_rpn_box_reg: 0.0363 (0.0746)  time: 0.5960  data: 0.0091  max me

/usr/local/lib/python3.7/dist-packages/pymongo/topology.py:165: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: https://pymongo.readthedocs.io/en/stable/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "
/usr/local/lib/python3.7/dist-packages/pymongo/topology.py:165: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: https://pymongo.readthedocs.io/en/stable/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "


Epoch: [2]  [   0/1397]  eta: 0:20:41  lr: 0.001000  loss: 1.2761 (1.2761)  loss_classifier: 0.2220 (0.2220)  loss_box_reg: 0.0226 (0.0226)  loss_objectness: 0.5362 (0.5362)  loss_rpn_box_reg: 0.4953 (0.4953)  time: 0.8889  data: 0.2109  max mem: 4553
Epoch: [2]  [ 100/1397]  eta: 0:11:59  lr: 0.001000  loss: 0.6707 (0.7443)  loss_classifier: 0.2262 (0.2281)  loss_box_reg: 0.0578 (0.0509)  loss_objectness: 0.3469 (0.3811)  loss_rpn_box_reg: 0.0315 (0.0843)  time: 0.5618  data: 0.0091  max mem: 4553
Epoch: [2]  [ 200/1397]  eta: 0:11:18  lr: 0.001000  loss: 0.5516 (0.7106)  loss_classifier: 0.1842 (0.2171)  loss_box_reg: 0.0202 (0.0473)  loss_objectness: 0.3126 (0.3706)  loss_rpn_box_reg: 0.0228 (0.0757)  time: 0.5811  data: 0.0096  max mem: 4553
Epoch: [2]  [ 300/1397]  eta: 0:10:29  lr: 0.001000  loss: 0.5088 (0.7074)  loss_classifier: 0.1674 (0.2104)  loss_box_reg: 0.0139 (0.0467)  loss_objectness: 0.3062 (0.3710)  loss_rpn_box_reg: 0.0191 (0.0794)  time: 0.5867  data: 0.0092  max me

/usr/local/lib/python3.7/dist-packages/pymongo/topology.py:165: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: https://pymongo.readthedocs.io/en/stable/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "
/usr/local/lib/python3.7/dist-packages/pymongo/topology.py:165: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: https://pymongo.readthedocs.io/en/stable/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "


Epoch: [3]  [   0/1397]  eta: 0:16:03  lr: 0.001000  loss: 0.6863 (0.6863)  loss_classifier: 0.1363 (0.1363)  loss_box_reg: 0.0355 (0.0355)  loss_objectness: 0.4094 (0.4094)  loss_rpn_box_reg: 0.1051 (0.1051)  time: 0.6898  data: 0.2237  max mem: 4553
Epoch: [3]  [ 100/1397]  eta: 0:12:34  lr: 0.001000  loss: 0.3812 (0.5008)  loss_classifier: 0.0999 (0.1230)  loss_box_reg: 0.0319 (0.0487)  loss_objectness: 0.2297 (0.2779)  loss_rpn_box_reg: 0.0243 (0.0512)  time: 0.5811  data: 0.0085  max mem: 4553
Epoch: [3]  [ 200/1397]  eta: 0:11:21  lr: 0.001000  loss: 0.5155 (0.5204)  loss_classifier: 0.1096 (0.1217)  loss_box_reg: 0.0340 (0.0472)  loss_objectness: 0.2955 (0.2902)  loss_rpn_box_reg: 0.0415 (0.0611)  time: 0.5933  data: 0.0090  max mem: 4553
Epoch: [3]  [ 300/1397]  eta: 0:10:23  lr: 0.001000  loss: 0.4024 (0.5465)  loss_classifier: 0.1075 (0.1233)  loss_box_reg: 0.0413 (0.0495)  loss_objectness: 0.2425 (0.3004)  loss_rpn_box_reg: 0.0266 (0.0733)  time: 0.5413  data: 0.0085  max me